In [79]:
import pandas as pd
import numpy as np
import requests
import re

In [80]:
URL='https://www.mobile.bg/obiavi/avtomobili-dzhipove'

In [81]:
div_regex=re.compile('<div')
div_close_regex=re.compile('</div>')
span_regex=re.compile('<span>.*?</span>')
div_class_info_regex=re.compile('<div class="info">\n.*?\n</div>')
div_price_regex=re.compile('<div>[0-9 ]+ лв.</div>')
div_model_regex=re.compile('<a href=\".*?\" class="title saveSlink">.*?</a>')
div_location_regex=re.compile('<div class="location">.*?</div>')
href_regex=re.compile('www.mobile.bg/obiava.*?"')
mp_label_regex=re.compile('<div class="mpLabel">.*?</div>')
mp_info_regex=re.compile('<div class="mpInfo">.*?</div>')
div_start_plus_end_regex=re.compile('<div>.*?</div>')
div_remover_regex=re.compile('</{0,1}(div|span|a).*?>')
span_title_regex=re.compile('<span class="Title">.*?</span>')
item_div_regex=re.compile('<div.*?>.+')


In [82]:

def extract_divs(text,searched_div):
    line_parsing=False
    result=''
    results=[]
    count=0
    for line in text.split('\n'):
        line=line.strip()
        #print(line,count)
        if line_parsing:
            if div_regex.search(line):
                count+=1
            if div_close_regex.search(line):
                count-=1

            result+=f"{line}\n"
            if count==0:
                line_parsing=False
                results.append(result)
                result=''

        else:
            if not line.startswith(searched_div) :
                continue

            line_parsing=True
            result+=f"{line}\n"
            count=1
    return results


In [83]:
def first(a):
    return '' if len(a)==0 else a[0]

In [84]:
def get_info_of_cars_on_page(r):
    cars_top=extract_divs(r.text,'<div class="item TOP " id="')
    cars_default=extract_divs(r.text,'<div class="item  " id="')
    cars_short=extract_divs(r.text,'<div id="shortList6"')
    cars=cars_top+cars_default+cars_short
    cars_params=[]
    cars_info=[]
    cars_prices=[]
    cars_models=[]
    individual_urls=[]
    locations=[]
    status=['TOP']*len(cars_top)+['Default']*len(cars_default)+['SHORT']*len(cars_short)
    for car in cars:
        cars_params.append(span_regex.findall(car))
        cars_info.append(first(div_class_info_regex.findall(car)))
        cars_prices.append(first(div_price_regex.findall(car)))
        cars_models.append(first(div_model_regex.findall(car)))
        locations.append(first(div_location_regex.findall(car)))

    for car_model in cars_models:
        individual_urls.append(href_regex.findall(car_model)[0][:-1])

    individual_urls
    return (cars_models,cars_info,cars_prices,cars_params,individual_urls,locations,status)



In [85]:


def get_individual_car_cards_info(car_url):
    individual=requests.get(f'https://{car_url}')
    individual.encoding = individual.apparent_encoding

    info_html=extract_divs(individual.text,'<div class="borderBox carParams">')[0]
    info_labels=mp_label_regex.findall(info_html)
    infos_details=mp_info_regex.findall(info_html)

    tech_html=extract_divs(individual.text,'<div class="techData">')[0]
    data=div_start_plus_end_regex.findall(tech_html)
    tech_labels=data[0::2]
    tech_details=data[1::2]

    additional=first(extract_divs(individual.text,'<div class="carExtri">'))
    titles=span_title_regex.findall(additional)
    items_htmls=extract_divs(div_close_regex.sub('\n</div>\n',div_regex.sub('\n<div',additional)),'<div class="items">')
    items=[]
    for item in items_htmls:
        items.append(item_div_regex.findall(item))
    return (info_labels,infos_details,tech_labels,tech_details,titles,items)



In [86]:
def clean_nested(source):
    source_patched=[]
    for nested1_source in source:
        nested1_source_patched=[]
        for nested2 in nested1_source:
            nested1_source_patched.append(div_remover_regex.sub('',nested2).strip())
        source_patched.append(nested1_source_patched)   
    return source_patched

In [87]:
def data_cleaner(func):
    def clean_cars_data(page_url):
        cars_models,cars_description,cars_prices,cars_params,individual_urls,infos_labels,infos_details,tech_labels,tech_details,locations,all_titles,all_items,status=func(page_url)
        cars_models_patched=[div_remover_regex.sub('',i).strip() for i in cars_models]
        cars_description_patched=[div_remover_regex.sub('',i).strip() for i in cars_description]
        cars_prices_patched=[div_remover_regex.sub('',i).strip() for i in cars_prices]
        locations_patched=[div_remover_regex.sub('',i).strip() for i in locations]

        cars_params_patched=clean_nested(cars_params)
        infos_labels_patched=clean_nested(infos_labels)
        infos_details_patched=clean_nested(infos_details)
        tech_labels_patched=clean_nested(tech_labels)
        tech_details_patched=clean_nested(tech_details)

        all_titles_patched=clean_nested(all_titles)
        all_items_patched=[clean_nested(i) for i in all_items]
        return [cars_models_patched,cars_description_patched,
                cars_prices_patched,cars_params_patched,
                individual_urls,infos_labels_patched,
                infos_details_patched,tech_labels_patched,
                tech_details_patched,locations_patched
                ,all_titles_patched,all_items_patched,status]
    return clean_cars_data


In [88]:
@data_cleaner
def get_cars_info(page_url):
    r=requests.get(page_url)
    r.encoding = r.apparent_encoding
    cars_models,cars_description,cars_prices,cars_params,individual_urls,locations,status=get_info_of_cars_on_page(r)
    infos_labels=[]
    infos_details=[]
    tech_labels=[]
    tech_details=[]
    all_titles=[]
    all_items=[]
    for url in individual_urls:
        info_label,info_detail,tech_label,tech_detail,titles,items=get_individual_car_cards_info(url)
        infos_labels.append(info_label)
        infos_details.append(info_detail)
        tech_labels.append(tech_label)
        tech_details.append(tech_detail)
        all_titles.append(titles)
        all_items.append(items)
    return [cars_models,cars_description,cars_prices,cars_params,individual_urls,infos_labels,infos_details,tech_labels,tech_details,locations,all_titles,all_items,status]


In [89]:
def get_cars_in_pages(pages):
    #cars_data=get_cars_info(URL)
    cars_data=get_cars_info(f'{URL}/p-{int(pages[0])}')
    for i in pages[1:]:
        cars=get_cars_info(f'{URL}/p-{int(i)}')
        for j in range(len(cars_data)):
            cars_data[j]=cars_data[j]  + cars[j]
    return cars_data

In [90]:
from concurrent.futures import ThreadPoolExecutor
def get_cars_in_pages_threaded(pages,threads):
    with ThreadPoolExecutor(threads) as pool:
        pages=np.array_split(pages,threads)
        result=[pool.submit(get_cars_in_pages,i) for i in pages]
        cars_data=result[0].result()
        for i in result[1:]:
            cars=i.result()
            for j in range(len(cars_data)):
                cars_data[j]=cars_data[j]  + cars[j]
    return cars_data

In [91]:
#cars_data=get_cars_in_pages(np.linspace(2,150,size,dtype=np.int64))
#cars_data=get_cars_in_pages(np.arange(2,10))
cars_data=get_cars_in_pages_threaded(np.arange(2,10),4)
cars_data

[['Audi A3 Реален Пробег ! Обслужен !',
  'Audi A3 SPORTBACK 1.4 TFSI 122hp',
  'Audi A3 Keyless Full led',
  'Audi A3 2.0 TFSI',
  'Audi A3 1.4i',
  'Audi A3 QUATTRO 3xS-LINE',
  'Audi A4 2, 0',
  'Audi A4 2.0d 140k 6ск италия',
  'Audi A4 1.9TDI 131кс. 6 скорости S-line',
  'Audi A4 1.9TDI-131 6SK',
  'Audi A4 2.0TDI-140к.с.-ИТАЛИЯ-6скорости-Sline-',
  'Audi A4 1.9TDI* REALNI KM*',
  'Audi A4 3, 2 Quatro ОБСЛУЖЕН, ТОП',
  'Audi A4',
  'Audi A4 2.0TDI НОВ ВНОС',
  'Audi A4',
  'Audi A4 2.0tdi 120ks 6sk!',
  'Audi A4 1.8 TFSI',
  'Audi A3 Mild Hybrid/3xSline/35TFSI/Limousine/Обслужена/B&O',
  'Audi A4 1.8 TFSI',
  'Audi A4 QUATTRO / DISTRONIC / NAVI',
  'Audi A4 2.0TDI NAVI* СОБСТВЕН ЛИЗИНГ* БАРТЕР',
  'Audi A4 1.8 S line',
  'Audi A4 2.0 TDi 143кс! 6ck!NAVI! Bi XENON !TOP !',
  'Audi A4 2.0TFSI* Кожа* QUATTRO*',
  'Audi A4 = 3.0TDI= S LINE= 245HP= НАВИГАЦИЯ =',
  'Audi A4 2.0TD-S-LINE QUATTRO',
  'Audi A4 163.000km-NEW-2.0TDI-150hp-NAVI-LED-AUTOMAT',
  'Audi A4 2.0 / 8ZF / Quattro',
 

In [92]:
def combine_labels(source):
    result=source[0]
    for labels in source[1:]:
        result+=labels
    return list(set(result))

In [93]:
all_tech_labels=combine_labels(cars_data[-6])
all_info_labels=combine_labels(cars_data[-8])
all_titles=combine_labels(cars_data[-3])
all_titles

['Защита', 'Безопасност', 'Интериор', 'Други', 'Комфорт', 'Екстериор']

In [94]:
def get_as_dicts(keys,values):
    result=[]
    for labels,data in zip(keys,values):
        current={}
        for label,value in zip(labels,data):
            current[label]=value
        result.append(current)
    return result

In [95]:
infos_as_dicts=get_as_dicts(cars_data[5],cars_data[6])
tech_as_dict=get_as_dicts(cars_data[7],cars_data[8])


In [96]:
additional_data_for_car=get_as_dicts(cars_data[10],cars_data[11])
additional_data_for_car

[{'Безопасност': ['GPS система за проследяване',
   'Автоматичен контрол на стабилността',
   'Адаптивни предни светлини',
   'Антиблокираща система',
   'Въздушни възглавници - Задни',
   'Въздушни възглавници - Предни',
   'Въздушни възглавници - Странични',
   'Електронна програма за стабилизиране',
   'Контрол на налягането на гумите',
   'Парктроник',
   'Система ISOFIX',
   'Система за динамична устойчивост',
   'Система за защита от пробуксуване',
   'Система за подпомагане на спирането'],
  'Други': ['Напълно обслужен', 'Нов внос', 'Сервизна книжка'],
  'Екстериор': ['4(5) Врати',
   'LED фарове',
   'Лети джанти',
   'Металик',
   'Халогенни фарове'],
  'Защита': ['Аларма', 'Централно заключване'],
  'Комфорт': ['Auto Start Stop function',
   'Bluetooth \\ handsfree система',
   'DVD, TV',
   'Steptronic, Tiptronic',
   'USB, audio\\video, IN\\AUX изводи',
   'Блокаж на диференциала',
   'Бордкомпютър',
   'Датчик за светлина',
   'Ел. Огледала',
   'Ел. Стъкла',
   'Климатрон

In [97]:

df=pd.DataFrame({'CarModel':cars_data[0],'CarDescription':cars_data[1],'CarPrice':cars_data[2],'Params':cars_data[3],'URL':cars_data[4],"Location":cars_data[9],'Status':cars_data[12]})
df=df.join(pd.DataFrame(tech_as_dict),how='outer').join(pd.DataFrame(infos_as_dicts),lsuffix='Tech',rsuffix='Info').join(pd.DataFrame(additional_data_for_car))
df


,CarModel,CarDescription,CarPrice,Params,URL,Location,Status,Дата на производствоTech,ДвигателTech,МощностTech,...,МощностInfo,ЕвростандартInfo,Скоростна кутияInfo,Пробег [км]Info,Безопасност,Други,Екстериор,Защита,Комфорт,Интериор
0,Audi A3 Реален Пробег ! Обслужен !,Нов Внос от Швейцария в безупречно техническо ...,23 999 лв.,"[декември 2016 г., 107 538 км, Черен, Дизелов,...",www.mobile.bg/obiava-11721377192980888-audi-a3...,"обл. София, гр. София",TOP,декември 2016,Дизелов,110 к.с.,...,110 к.с.,Евро 6,Автоматична,107538 км,"[GPS система за проследяване, Автоматичен конт...","[Напълно обслужен, Нов внос, Сервизна книжка]","[4(5) Врати, LED фарове, Лети джанти, Металик,...","[Аларма, Централно заключване]","[Auto Start Stop function, Bluetooth \ handsfr...",NaN
1,Audi A3 SPORTBACK 1.4 TFSI 122hp,"Продавам AUDI A3, SPORTBACK 1. 4 TFSI, бензин ...",24 500 лв.,"[октомври 2013 г., 204 000 км, Сив, Бензинов, ...",www.mobile.bg/obiava-11718217748914834-audi-a3...,"обл. София, гр. София",TOP,октомври 2013,Бензинов,122 к.с.,...,122 к.с.,Евро 5,Автоматична,204000 км,"[GPS система за проследяване, Антиблокираща си...","[Напълно обслужен, С регистрация]","[4(5) Врати, Ксенонови фарове, Лети джанти, Ме...","[Аларма, Каско, Централно заключване]","[Auto Start Stop function, Bluetooth \ handsfr...",[Кожен салон]
2,Audi A3 Keyless Full led,"Keyless, Full led, ксенон, подгрев, преден и з...",25 000 лв.,"[ноември 2015 г., 201 000 км, Бял, Дизелов, 15...",www.mobile.bg/obiava-11697480660026130-audi-a3...,"обл. Бургас, гр. Поморие",TOP,ноември 2015,Дизелов,150 к.с.,...,150 к.с.,Евро 6,Автоматична,201000 км,"[GPS система за проследяване, Адаптивни предни...","[Лизинг, Напълно обслужен, Нов внос, С регистр...","[4(5) Врати, LED фарове, Ксенонови фарове, Лет...","[Аларма, Каско, Централно заключване]","[Auto Start Stop function, Bluetooth \ handsfr...",[Кожен салон]
3,Audi A3 2.0 TFSI,"Обслужен на 144500км , сменени 4 нови дискове ...",32 900 лв.,"[юли 2017 г., 145 380 км, Бял, Бензинов, 186 к...",www.mobile.bg/obiava-11701528856658093-audi-a3...,"обл. София, гр. София",TOP,юли 2017,Бензинов,186 к.с.,...,186 к.с.,Евро 5,Автоматична,145380 км,"[Адаптивни предни светлини, Въздушни възглавни...","[Напълно обслужен, Нов внос]","[4(5) Врати, LED фарове, Ксенонови фарове, Лет...","[Аларма, Каско, Централно заключване]","[Auto Start Stop function, Bluetooth \ handsfr...",NaN
4,Audi A3 1.4i,Колата е в отлично състояние реални километри ...,34 900 лв.,"[септември 2017 г., 200 000 км, Черен, Бензино...",www.mobile.bg/obiava-11686487589024041-audi-a3...,"обл. София, гр. София",TOP,септември 2017,Бензинов,150 к.с.,...,150 к.с.,Евро 6,Ръчна,200000 км,"[Антиблокираща система, Въздушни възглавници -...","[Нов внос, Сервизна книжка]","[4(5) Врати, LED фарове, Ксенонови фарове, Лет...","[Аларма, Централно заключване]","[Auto Start Stop function, Bluetooth \ handsfr...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Audi A8 4.2TDI* MATRIX* SOFTCLOSE* BOSE* KEYLESS,"Здравейте, продавам Аудито от снимките 4х4 qua...",26 900 лв.,"[януари 2013 г., 297 655 км, Бял, Дизелов, 351...",www.mobile.bg/obiava-11722101555428416-audi-a8...,"обл. Русе, гр. Бяла",TOP,януари 2013,Дизелов,351 к.с.,...,351 к.с.,Евро 5,Автоматична,297655 км,NaN,NaN,NaN,NaN,NaN,NaN
148,Audi A8 4.2TDI/FULL LED/DISTRON/CAMERA/ПОДГРEB...,! ! ! СОБСТВЕН ЛИЗИНГ ! ! ! БЕЗ ДОКАЗВАНЕ НА ...,32 000 лв.,"[януари 2012 г., 189 300 км, Черен, Дизелов, 3...",www.mobile.bg/obiava-11718624679348570-audi-a8...,"обл. София, гр. София",TOP,януари 2012,Дизелов,351 к.с.,...,351 к.с.,Евро 5,Автоматична,189300 км,"[GPS система за проследяване, Автоматичен конт...","[4x4, Buy back, Бартер, Лизинг, Нов внос, С ре...","[4(5) Врати, LED фарове, Лети джанти, Металик,...","[Аларма, Централно заключване]","[Auto Start Stop function, Bluetooth \ handsfr...",NaN
149,Audi A8 * 4.2TDI* FULL-LED* NIGHT-VISION* INDI...,"Здравейте , продаваме посочения автомобил. Отл...",32 900 лв.,"[декем

In [98]:
for label in all_info_labels:
    df_copy=df.dropna(subset=[f"{label}Tech",f"{label}Info"])
    if (df_copy[f"{label}Tech"]!=df_copy[f"{label}Info"]).sum():
        print(label)

In [99]:
df.columns

Index(['CarModel', 'CarDescription', 'CarPrice', 'Params', 'URL', 'Location',
       'Status', 'Дата на производствоTech', 'ДвигателTech', 'МощностTech',
       'ЕвростандартTech', 'Кубатура [куб.см]', 'Скоростна кутияTech',
       'Категория', 'Пробег [км]Tech', 'Цвят', 'VIN номер',
       'Дата на производствоInfo', 'ДвигателInfo', 'МощностInfo',
       'ЕвростандартInfo', 'Скоростна кутияInfo', 'Пробег [км]Info',
       'Безопасност', 'Други', 'Екстериор', 'Защита', 'Комфорт', 'Интериор'],
      dtype='object')

In [100]:
#df.to_csv('out_more.csv')

: 